In [3]:
!pip install gradio langchain langchain_groq langchain_community faiss-cpu Pillow pytesseract pdf2image PyPDF2 sentence_transformers groq tiktoken
!apt-get update
!apt-get install -y poppler-utils tesseract-ocr

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://cli.github.com/packages stable InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

Mount Google Drive

In [4]:
from google.colab import drive
import os

drive.mount('/content/drive')

drive_base = "/content/drive/MyDrive/BioChatbot"
pdf_folder = os.path.join(drive_base, "pdfs")
img_folder = os.path.join(drive_base, "pdf_images")
os.makedirs(pdf_folder, exist_ok=True)
os.makedirs(img_folder, exist_ok=True)

print(f"Data will be saved in Google Drive under: {drive_base}")


Mounted at /content/drive
Data will be saved in Google Drive under: /content/drive/MyDrive/BioChatbot


Upload PDFs into Drive

In [5]:
from google.colab import files
uploaded = files.upload()

for filename, content in uploaded.items():
    destination_path = os.path.join(pdf_folder, filename)
    with open(destination_path, "wb") as f:
        f.write(content)
    print(f"Saved to Drive: {destination_path}")

print("All PDFs saved permanently in Google Drive.")

✅ All PDFs saved permanently in Google Drive.


OCR & Extract Text

In [6]:
from pdf2image import convert_from_path
import pytesseract
from PyPDF2 import PdfReader
from multiprocessing import Pool, cpu_count

def process_page(args):
    pdf_path, filename, page_num = args
    try:
        image = convert_from_path(pdf_path, dpi=150, first_page=page_num, last_page=page_num)[0]
        image_path = os.path.join(img_folder, f"{filename}_page_{page_num}.png")
        image.save(image_path, "PNG")
        text = pytesseract.image_to_string(image)
        return f"\n\n--- Page {page_num} of {filename} ---\n{text}"
    except Exception as e:
        return f"\n\n--- Page {page_num} of {filename} ---\nERROR: {e}"

def ocr_pdfs_in_drive(pdf_folder, output_file):
    tasks = []
    for filename in sorted(os.listdir(pdf_folder)):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            try:
                reader = PdfReader(pdf_path)
                total_pages = len(reader.pages)
                print(f"{filename} → {total_pages} pages")
                for page_num in range(1, total_pages + 1):
                    tasks.append((pdf_path, filename, page_num))
            except Exception as e:
                print(f"Could not read {filename}: {e}")

    with Pool(cpu_count()) as pool:
        for result in pool.imap_unordered(process_page, tasks):
            with open(output_file, "a", encoding="utf-8") as f:
                f.write(result)

    print(f"OCR completed. Output saved to {output_file}")

ocr_text_file = os.path.join(drive_base, "ocr_output.txt")
ocr_pdfs_in_drive(pdf_folder, ocr_text_file)

with open(ocr_text_file, "r", encoding="utf-8") as f:
    ocr_text = f.read()

full_text_path = os.path.join(drive_base, "biology_full_content.txt")
with open(full_text_path, "w", encoding="utf-8") as f:
    f.write(ocr_text)

print(" Text saved in Google Drive.")


Unit 01-Introduction to Biology-English.pdf → 5 pages
Unit 02-Chemical and cellular basis of life-English.pdf → 66 pages
Unit 03-Evolution and diversity of organisms-English.pdf → 35 pages
Unit 04-Plant form and function-English.pdf → 49 pages
Unit 05 Part 1-Animal form and function-English.pdf → 80 pages
Unit 05 Part 2-Animal form and function-English.pdf → 88 pages
Unit 06-Genetics-English.pdf → 43 pages
Unit 07-Molecular Biology and Recombinant DNA Technology-English.pdf → 61 pages
Unit 08-Environmental Biology-English.pdf → 43 pages
Unit 09-Microbiology-English.pdf → 37 pages
Unit 10-Applied Biology-English.pdf → 30 pages
✅ OCR completed. Output saved to /content/drive/MyDrive/BioChatbot/ocr_output.txt
✅ Text saved in Google Drive.


Split & Create Vector Store

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.create_documents([ocr_text])

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding_model)

/tmp/ipython-input-4051102128.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Save FAISS index

In [8]:
vectorstore.save_local(os.path.join(drive_base, "faiss_index"))
print(" FAISS index saved in Google Drive.")

✅ FAISS index saved in Google Drive.


Connect LLM + QA

In [17]:
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

os.environ["GROQ_API_KEY"] = "your_groq_api_key_here"
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.2)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

print(" Biology Chatbot Ready!")

✅ Biology Chatbot Ready!


Gradio Chat UI

In [18]:
import gradio as gr

def chat_with_bot(user_input, history):
    try:
        answer = qa.run(user_input)
        return answer
    except Exception as e:
        return f"Error: {str(e)}"

demo = gr.ChatInterface(
    fn=chat_with_bot,
    title="A/L Biology Chatbot",
    description="Ask any A/L Biology Questions.",
    theme="default",
)

demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://feb71160f622c22fe4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


After Restart

In [20]:
!pip install gradio langchain langchain_groq langchain_community faiss-cpu sentence-transformers groq tiktoken

from google.colab import drive
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

drive.mount('/content/drive')
drive_base = "/content/drive/MyDrive/BioChatbot"

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local(
    os.path.join(drive_base, "faiss_index"),
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

print(" FAISS index loaded from Drive")

from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

os.environ["GROQ_API_KEY"] = "your_groq_api_key_here"
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.2)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

print(" Biology Chatbot Ready (Reloaded)")

import gradio as gr

def chat_with_bot(user_input, history):
    try:
        answer = qa.run(user_input)
        return answer
    except Exception as e:
        return f"Error: {str(e)}"

demo = gr.ChatInterface(
    fn=chat_with_bot,
    title="A/L Biology Chatbot",
    description="Ask any A/L Biology Questions.",
    theme="default",
)

demo.launch(share=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ FAISS index loaded from Drive
✅ Biology Chatbot Ready (Reloaded)


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://868eaf36a5df27144b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Test Process

In [27]:
test_questions = [
    {"q": "What is photosynthesis?",
     "a": "The process by which green plants use sunlight to synthesize food from carbon dioxide and water."},
    {"q": "Name the organelle responsible for ATP production.",
     "a": "Mitochondria"},
    {"q": "What are the stages of mitosis?",
     "a": "Prophase, metaphase, anaphase, and telophase"},
]



Semantic Accuracy

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_similarity(bot_answer, expected_answer):
    emb_bot = embedding_model.embed_query(bot_answer)
    emb_expected = embedding_model.embed_query(expected_answer)
    return cosine_similarity([emb_bot], [emb_expected])[0][0]

scores = []
for item in test_questions:
    bot_answer = qa.run(item["q"])
    sim = evaluate_similarity(bot_answer, item["a"])
    scores.append(sim)
    print(f"\nQ: {item['q']}")
    print(f"Bot: {bot_answer}")
    print(f"Expected: {item['a']}")
    print(f"Similarity Score: {sim:.2f}")

print(f"\n Average Similarity Score: {sum(scores)/len(scores):.2f}")



Q: What is photosynthesis?
Bot: According to the provided context, photosynthesis is a metabolic process by which light energy is trapped and converted to chemical energy. Chemical energy is stored in chemical bonds of carbohydrates, fats, oils, and proteins.
Expected: The process by which green plants use sunlight to synthesize food from carbon dioxide and water.
Similarity Score: 0.61

Q: Name the organelle responsible for ATP production.
Bot: According to the provided context, the organelle responsible for ATP production is the mitochondria. Specifically, it is stated that one of the functions of mitochondria is to "Synthesize ATP in aerobic respiration".
Expected: Mitochondria
Similarity Score: 0.63

Q: What are the stages of mitosis?
Bot: According to the provided context, the stages of mitosis are:

1. Prophase
2. Prometaphase
3. Metaphase
4. Anaphase
5. Telophase
Expected: Prophase, metaphase, anaphase, and telophase
Similarity Score: 0.66

 Average Similarity Score: 0.64


LLM-Based Grading (LangChain Eval)

In [29]:
from langchain.evaluation.qa import QAEvalChain

examples = [{"query": t["q"], "answer": t["a"]} for t in test_questions]
predictions = [{"query": t["q"], "result": qa.run(t["q"])} for t in test_questions]

eval_chain = QAEvalChain.from_llm(llm)
graded = eval_chain.evaluate(examples, predictions)

for i, eg in enumerate(examples):
    print(f"\nQ: {eg['query']}")
    print(f"Correct: {eg['answer']}")
    print(f"Bot: {predictions[i]['result']}")
    print(f"Eval: {graded[i]['results']}")



Q: What is photosynthesis?
Correct: The process by which green plants use sunlight to synthesize food from carbon dioxide and water.
Bot: According to the provided context, photosynthesis is a metabolic process by which light energy is trapped and converted to chemical energy. Chemical energy is stored in chemical bonds of carbohydrates, fats, oils, and proteins.
Eval: GRADE: CORRECT

Q: Name the organelle responsible for ATP production.
Correct: Mitochondria
Bot: According to the provided context, the organelle responsible for ATP production is the mitochondria. Specifically, it is stated that one of the functions of mitochondria is to "Synthesize ATP in aerobic respiration".
Eval: GRADE: CORRECT

Q: What are the stages of mitosis?
Correct: Prophase, metaphase, anaphase, and telophase
Bot: According to the provided context, the stages of mitosis are:

1. Prophase
2. Prometaphase
3. Metaphase
4. Anaphase
5. Telophase
Eval: GRADE: CORRECT
